# Features embedding
Here we would embedd all the features that need to be feed into the model from the JFK_2018_2022_final_features.csv file created using Feature extraction.ipynb file. 
We have used different type embedding for different type of features. 
Features like "Delaym4h_s",'Temperature_F', 'Dew_Point_F',  'Humidity_%',  'Wind_Speed_mph', 'Wind_Gust_mph', 'Pressure_in'and 'Precip_in' have values ranging upto 2 order of magnitudes. These features are normalised using min-max scaler. 

Features like 'DayOfWeek', "Month", "Hour" and "wind direction"  has inherent periodicty. For example hour 24 and hour 1 are closer to each other then hour 1 and 3. For these features one-hot encoding does not preserve the neigbourhood relationship these numbers have. Therefore we have used cycic embedding, where these features are mapped to a circle using x and y component. 
Finally feature "condition" is catagorical variable and has to be one hot encoded.


In [1]:
import pandas as pd
import imblearn
import datetime
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import os
from scipy.interpolate import interp1d
import seaborn as sns
sns.set_style("whitegrid")
from datetime import datetime, timedelta

In [2]:
feature_df=pd.read_csv("JFK_2018_2022_final_features_prior.csv")

feature_df.Scheduled_Dep_Time = pd.to_datetime(feature_df.Scheduled_Dep_Time)
feature_df= feature_df.set_index('Scheduled_Dep_Time')
feature_df

,DayOfWeek,Month,Hour,Temperature_F,Dew_Point_F,Humidity_%,Wind_Speed_mph,Wind_Gust_mph,Pressure_in,Precip_in,Wind,Condition,Carrier_delay_probabilty,Destination_delay_probabilty,Delaym4h_s,Yesterday_delay_probabilty,DepDelay,Label
Scheduled_Dep_Time,,,,,,,,,,,,,,,,,,
2018-01-01 05:30:00,1,1,5,7.0,-5.0,58.0,17.0,0.0,30.34,0.0,NW,Fair,0.270596,0.222359,0.0,NaN,-9.0,0.0
2018-01-01 05:30:00,1,1,5,7.0,-5.0,58.0,21.0,0.0,30.36,0.0,NW,Fair / Windy,0.270596,0.291925,0.0,NaN,67.0,1.0
2018-01-01 05:45:00,1,1,5,7.0,-5.0,58.0,16.0,0.0,30.35,0.0,NW,Fair,0.270596,0.214052,0.0,NaN,-3.0,0.0
2018-01-01 05:50:00,1,1,5,7.0,-5.0,58.0,21.0,0.0,30.36,0.0,NW,Fair / Windy,0.270596,0.198739,0.0,NaN,83.0,1.0
2018-01-01 06:00:00,1,1,6,8.0,-5.0,55.0,18.0,0.0,30.37,0.0,NW,Fair,0.270596,0.186808,0.0,NaN,103.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-31 22:50:00,7,7,22,75.0,68.0,79.0,10.0,0.0,30.05,0.0,SSW,Light Rain,0.207507,0.254032,9.0,0.217391,-1.0,0.0
2022-07-31 22:55:00,7,7,22,75.0,59.0,57.0,5.0,0.0,30.14,0.0,NE,Fair,0.517304,0.299375,9.0,0.217391,29.0,1.0
2022-07-31 22:59:00,7,7,22,75.0,68.0,79.0,10.0,0.0,30.05,0.0,SSW,Light Rain,0.207507,0.094340,8.0,0.217391,-16.0,0.0


In [3]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
# from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


def cyclic_transform(t,T_max):
    return np.sin(2*t*np.pi/T_max),np.cos(2*t*np.pi/T_max)
embedded_df=feature_df.copy()

embedded_df=embedded_df.dropna()

#embedding wind direction
dir_keys=  ["N","NNE","NE", "ENE", "E", "ESE", "SE", "SSE","S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW",'CALM','VAR']
dir_values=np.arange(0,2*np.pi+0.001,np.pi/16)      
dir_x=np.append(np.cos(dir_values),[0,0.25])
dir_y=np.append(np.sin(dir_values),[0,0.25])
wind_dict_x,wind_dict_y={},{}
for k, vx,vy in zip(dir_keys,dir_x,dir_y):
    wind_dict_x[k]=(vx)
    wind_dict_y[k]=(vy)
embedded_df["Wind_x"]=embedded_df["Wind"].map(wind_dict_x)
embedded_df["Wind_y"]=embedded_df["Wind"].map(wind_dict_y) 
embedded_df.drop("Wind",axis=1, inplace=True)




norm_cols=["Delaym4h_s",'Temperature_F', 'Dew_Point_F',  'Humidity_%',  'Wind_Speed_mph', 'Wind_Gust_mph', 'Pressure_in','Precip_in']
scaler = MinMaxScaler()
embedded_df[norm_cols] = scaler.fit_transform(
    embedded_df[norm_cols])

cyclic_cols={'DayOfWeek':7, "Month":12,"Hour":24}
for keys in cyclic_cols.keys():
    embedded_df[keys+"_x"],embedded_df[keys+"_y"]=cyclic_transform(embedded_df[keys],cyclic_cols[keys])
    embedded_df.drop(keys, axis=1, inplace=True)

   
    

hot_cols=["Condition" ]
transformer = make_column_transformer(( OneHotEncoder(), hot_cols),remainder='passthrough')

transformed = transformer.fit_transform(embedded_df)
print(transformer.get_feature_names())
embedded_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())


['onehotencoder__x0_Cloudy', 'onehotencoder__x0_Cloudy / Windy', 'onehotencoder__x0_Drizzle and Fog', 'onehotencoder__x0_Fair', 'onehotencoder__x0_Fair / Windy', 'onehotencoder__x0_Fog', 'onehotencoder__x0_Fog / Windy', 'onehotencoder__x0_Haze', 'onehotencoder__x0_Haze / Windy', 'onehotencoder__x0_Heavy Rain', 'onehotencoder__x0_Heavy Rain / Windy', 'onehotencoder__x0_Heavy Snow', 'onehotencoder__x0_Heavy T-Storm', 'onehotencoder__x0_Heavy T-Storm / Windy', 'onehotencoder__x0_Light Drizzle', 'onehotencoder__x0_Light Drizzle / Windy', 'onehotencoder__x0_Light Freezing Drizzle', 'onehotencoder__x0_Light Freezing Rain', 'onehotencoder__x0_Light Rain', 'onehotencoder__x0_Light Rain / Windy', 'onehotencoder__x0_Light Rain with Thunder', 'onehotencoder__x0_Light Sleet', 'onehotencoder__x0_Light Sleet / Windy', 'onehotencoder__x0_Light Snow', 'onehotencoder__x0_Light Snow / Windy', 'onehotencoder__x0_Light Snow and Sleet', 'onehotencoder__x0_Light Snow and Sleet / Windy', 'onehotencoder__x0_M

C:\Users\lalit\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [4]:
print(transformer.get_feature_names())
transformed_df = pd.DataFrame(transformed, columns=transformer.get_feature_names())
transformed_df.head()
print(len(transformed_df))
print(len(transformed[0]))

['onehotencoder__x0_Cloudy', 'onehotencoder__x0_Cloudy / Windy', 'onehotencoder__x0_Drizzle and Fog', 'onehotencoder__x0_Fair', 'onehotencoder__x0_Fair / Windy', 'onehotencoder__x0_Fog', 'onehotencoder__x0_Fog / Windy', 'onehotencoder__x0_Haze', 'onehotencoder__x0_Haze / Windy', 'onehotencoder__x0_Heavy Rain', 'onehotencoder__x0_Heavy Rain / Windy', 'onehotencoder__x0_Heavy Snow', 'onehotencoder__x0_Heavy T-Storm', 'onehotencoder__x0_Heavy T-Storm / Windy', 'onehotencoder__x0_Light Drizzle', 'onehotencoder__x0_Light Drizzle / Windy', 'onehotencoder__x0_Light Freezing Drizzle', 'onehotencoder__x0_Light Freezing Rain', 'onehotencoder__x0_Light Rain', 'onehotencoder__x0_Light Rain / Windy', 'onehotencoder__x0_Light Rain with Thunder', 'onehotencoder__x0_Light Sleet', 'onehotencoder__x0_Light Sleet / Windy', 'onehotencoder__x0_Light Snow', 'onehotencoder__x0_Light Snow / Windy', 'onehotencoder__x0_Light Snow and Sleet', 'onehotencoder__x0_Light Snow and Sleet / Windy', 'onehotencoder__x0_M

In [5]:
embedded_df.to_csv("Embedded_features_2018_2022_prior.csv")

In [6]:
embedded_df.columns

Index(['onehotencoder__x0_Cloudy', 'onehotencoder__x0_Cloudy / Windy',
       'onehotencoder__x0_Drizzle and Fog', 'onehotencoder__x0_Fair',
       'onehotencoder__x0_Fair / Windy', 'onehotencoder__x0_Fog',
       'onehotencoder__x0_Fog / Windy', 'onehotencoder__x0_Haze',
       'onehotencoder__x0_Haze / Windy', 'onehotencoder__x0_Heavy Rain',
       'onehotencoder__x0_Heavy Rain / Windy', 'onehotencoder__x0_Heavy Snow',
       'onehotencoder__x0_Heavy T-Storm',
       'onehotencoder__x0_Heavy T-Storm / Windy',
       'onehotencoder__x0_Light Drizzle',
       'onehotencoder__x0_Light Drizzle / Windy',
       'onehotencoder__x0_Light Freezing Drizzle',
       'onehotencoder__x0_Light Freezing Rain', 'onehotencoder__x0_Light Rain',
       'onehotencoder__x0_Light Rain / Windy',
       'onehotencoder__x0_Light Rain with Thunder',
       'onehotencoder__x0_Light Sleet',
       'onehotencoder__x0_Light Sleet / Windy', 'onehotencoder__x0_Light Snow',
       'onehotencoder__x0_Light Snow / Wi

In [7]:
df_19_22_pr=pd.read_csv("Embedded_features_2018_2022_prior.csv")
df_22_post=pd.read_csv("Embedded_features_Aug2022_posterior.csv")
df_all=pd.read_csv("Embedded_features_2018_2022.csv")

In [8]:
df_all=df_all['2022-08-01':'2022-08-31'].iloc[]
df_all.Carrier_delay_probabilty.hist(density=True)
df_22_post.Carrier_delay_probabilty.hist(alpha=.5,
                                         density=True)


SyntaxError: invalid syntax (Temp/ipykernel_17436/989252039.py, line 1)

In [ ]:
df_all[-10715  :-1].Destination_delay_probabilty.hist()

In [ ]:
df_22_post.iloc[-10715  :-1].Destination_delay_probabilty.hist()

In [ ]:
df_19_22_pr.Destination_delay_probabilty.hist()

In [ ]:
df_22_post.Month_x

In [ ]:
df_19_22_pr.Month_x.unique()

In [ ]:
\